# Analyse de KTlO2

In [2]:
import numpy as np
from pymatgen.core.operations import SymmOp
from mp_api.client import MPRester

In [3]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [4]:
mp_key = "txVH20lqfEIM4Yxb2k5ptbih3BpgcIsN"
mp_id = "mp-8175"

In [5]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
    inst = SpacegroupAnalyzer(prim_struc)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Visualisation de la structure

In [6]:
i_atom = 7
i_symmop = 2

In [10]:
view = quick_view(prim_struc, "packed", conventional = True)

display(view)

view.script('select potassium; color white')
view.script('select thallium; color pink')
view.script('select oxygen; color palegreen')


JsmolView(layout=Layout(align_self='stretch', height='400px'))

## Maille crystalline

In [13]:
maille = inst.get_primitive_standard_structure()

print(maille)

Full Formula (K1 Tl1 O2)
Reduced Formula: KTlO2
abc   :   6.419731   6.419731   6.419731
angles:  31.206844  31.206844  31.206844
pbc   :       True       True       True
Sites (4)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  K     0.5       0.5       0.5
  1  Tl    0         0         0
  2  O     0.732084  0.732084  0.732084
  3  O     0.267916  0.267916  0.267916


On voit que les données ci dessus correspondent bien au dessin que l'on voit plus haut, c'est-à-dire que les coordonées réduites sont: \
 Le Thallium (en rose) se trouve aux coordonnées \begin{pmatrix}0 & 0 & 0\end{pmatrix} Le potassium (en blanc) se trouve aux coordonnées \begin{pmatrix}0.5 & 0.5 & 0.5\end{pmatrix}  Les deux atomes d'oxygen (en vert) se trouvent respactivement aux coordonées \begin{pmatrix}0.732084 & 0.732084 & 0.732084\end{pmatrix} et \begin{pmatrix}0.267916 & 0.267916 & 0.267916\end{pmatrix} 
 
Les trois vecteurs de base $\begin{pmatrix} a & b & c\end{pmatrix} $sont de longueur $\begin{pmatrix} 6.419731  & 6.419731 &  6.419731\end{pmatrix} \mathring{A}$

# Vecteurs de bases des réseaux direct et réciproque

In [82]:
# reseau direct

lattice = prim_struc.lattice
print("Les vecteurs de base du réseau direct :\n")
print(lattice.matrix)



Les vecteurs de base du réseau direct :

[[ 3.31534541 -0.0077329   5.49739644]
 [ 1.52349981  2.94457495  5.49739644]
 [-0.01274142 -0.0077329   6.41971368]]


Les vecteurs de base du réseau direct sont  : <div align="center">$\begin{align*}\vec{a}&=a\cdot \begin{pmatrix}3.31534541 & -0.0077329  & 5.49739644\end{pmatrix}\\\vec{b}&=b\cdot \begin{pmatrix}1.52349981  & 2.94457495 & 5.49739644\end{pmatrix}\\\vec{c}&=c\cdot \begin{pmatrix}-0.01274142 & -0.0077329  & 6.41971368 \end{pmatrix}\end{align*}$</div>


In [83]:
#reseau réciproque

reciprocal_lattice = lattice.reciprocal_lattice
print("Les vecteurs de base du réseau réciproque : \n")
print(reciprocal_lattice.matrix)

Les vecteurs de base du réseau réciproque : 

[[ 1.88863811e+00 -9.81956342e-01  2.56562240e-03]
 [ 7.10980402e-04  2.12865988e+00  2.56549960e-03]
 [-1.61790720e+00 -9.81957181e-01  9.74339014e-01]]


Dans notre cours nous avons vu que les vecteurs de base du réseau réciproque pouvaient être obtenus grâce aux formules suivantes : <div align="center">$\begin{equation*}\vec{a}^* = \dfrac{2\pi}{\Omega}(\vec{b}\times\vec{c}) \end{equation*}$</div>

<div align="center">$\begin{equation*}\vec{b}^* = \dfrac{2\pi}{\Omega}(\vec{c}\times\vec{a})\end{equation*}$</div>

<div align="center">$\begin{equation*}\vec{c}^* = \dfrac{2\pi}{\Omega}(\vec{a}\times\vec{b})\end{equation*}$</div>

Les résultats obtenus vérifient bien cette relation : <div align="center">$\begin{equation*}\vec{a}^* = \begin{pmatrix} 1.88863811e+00 & -9.81956342e-01 & 2.56562240e-03 \end{pmatrix} \end{equation*}$</div>

<div align="center">$\begin{equation*}\vec{b}^* = \begin{pmatrix}7.10980402e-04 & 2.12865988e+00 & 2.56549960e-03\end{pmatrix}\end{equation*}$</div>

<div align="center">$\begin{equation*}\vec{c}^* = \begin{pmatrix}-1.61790720e+00 &-9.81957181e-01 & 9.74339014e-01\end{pmatrix}\end{equation*}$</div>

# Type de maille et système crystallin

In [84]:
lattice_type = inst.get_lattice_type()

crystal_system = inst.get_crystal_system()

print("Type de maille :", lattice_type)
print("Système crystallin :", crystal_system)

Type de maille : rhombohedral
Système crystallin : trigonal


On voit que le système est trigonal, cela est cohérent étant donné que nous avons $a=b=c= 6.419731\mathring{A} $ et $\alpha=\beta=\gamma=31.206844° \neq 90°$

On peut alors calculer le volume de la maille par:<div align="center">$\begin{align*}\Omega &= \vec{a}\cdot(\vec{b}\times\vec{c})\\&=a^3\cdot\sqrt{1-3\cos^2\alpha+2\cos^3\alpha}\\&=63\mathring{A}^3\end{align*}$</div>

Mais également le volume de la maille dans le réseau réciproque grâce à la relation théorique qui lie les deux : : <div align="center">$\begin{align*}\Omega^* &= \vec{a}^*\cdot(\vec{b}^*\times\vec{c}^*)\\& = \dfrac{(2\pi)^3}{\Omega} \\&=3.93730\mathring{A}^3\end{align*}$</div>

# Groupe Ponctuel

In [16]:
point_group = inst.get_point_group_symbol()

print ("Groupe ponctuel:", point_group)

Groupe ponctuel: -3m


Le groupe spatial du $KTlO_2$ est $R\overline3m$ ce qui signifie que l'on aura un axe de roto-inversion d'ordre 3 le long de [111] ainsi que des plans miroirs le long des directions $[1\overline10]$, $[01\overline1]$ et $[\overline101]$